In [ ]:
# Importing necessary libraries
import requests  # For making HTTP requests to fetch webpage data
from bs4 import BeautifulSoup  # For parsing and extracting content from HTML
from tqdm import tqdm  # For showing progress bars in loops

# A dictionary containing the number of Sargas (chapters) in each Kanda (book) of the Valmiki Ramayana
kanda_sarga_map = {
    1: 77,   # Bala Kanda
    2: 119,  # Ayodhya Kanda
    3: 75,   # Aranya Kanda
    4: 67,   # Kishkindha Kanda
    5: 68,   # Sundara Kanda
    6: 131   # Yuddha Kanda
}

# Base URL with placeholders to be formatted with Kanda and Sarga values
base_url = "https://www.valmiki.iitk.ac.in/sloka?field_kanda_tid={}&language=dv&field_sarga_value={}"

# Output file where the extracted content will be saved
output_file = "valmiki_ramayana1.txt"

# Opening the output file in write mode with UTF-8 encoding
with open(output_file, "w", encoding="utf-8") as file:

    # Loop through each Kanda in the dictionary
    for kanda, total_sargas in kanda_sarga_map.items():
        print(f"\n📖 Starting Kanda {kanda} having {total_sargas} Sargas")

        # Initialize a progress bar for the current Kanda
        pbar = tqdm(total=total_sargas, desc=f"Kanda {kanda}", unit='sarga', position=0, leave=True)

        # Loop through all Sargas in the current Kanda
        for sarga in range(1, total_sargas + 1):
            # Format the URL for the current Kanda and Sarga
            url = base_url.format(kanda, sarga)
            # Send a GET request to the webpage
            response = requests.get(url)

            # Initialize a list to store the extracted sloka lines
            sarga_lines = []

            # If the request was successful
            if response.status_code == 200:
                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.content, "html.parser")
                # Find all relevant rows containing sloka data
                rows = soup.find_all("div", class_="views-row")

                # Extract the Sanskrit sloka text from each row
                for row in rows:
                    text_div = row.find("div", class_="views-field views-field-field-htetrans")
                    if text_div:
                        text = text_div.get_text(strip=True)  # Remove leading/trailing spaces
                        sarga_lines.append(text)

                # Write the current Sarga to the file if it contains valid lines
                if sarga_lines:
                    file.write(f"\n\n### Kanda {kanda} - Sarga {sarga} ###\n")
                    file.write("\n".join(sarga_lines) + "\n")

            else:
                # If the request failed, print an error message
                print(f"❌ Failed to fetch Kanda {kanda} - Sarga {sarga}")

            # Update the progress bar
            pbar.update(1)

        # Close the progress bar after the Kanda is done
        pbar.close()
        print(f"✅ Completed Kanda {kanda} successfully!\n")

# Print completion message after all Kandams are scraped
print("\n🎉 Completed fetching Valmiki Ramayana!")
print(f"Data saved to: {output_file}")



📖 Starting Kanda 1 having 77 Sargas


Kanda 1: 100%|██████████████████████████████████████████████████████████████████████| 77/77 [00:49<00:00,  1.54sarga/s]


✅ Completed Kanda 1 successfully!


📖 Starting Kanda 2 having 119 Sargas


Kanda 2: 100%|████████████████████████████████████████████████████████████████████| 119/119 [01:34<00:00,  1.26sarga/s]


✅ Completed Kanda 2 successfully!


📖 Starting Kanda 3 having 75 Sargas


...

✅ Completed Kanda 4 successfully!


📖 Starting Kanda 5 having 68 Sargas


Kanda 5: 100%|██████████████████████████████████████████████████████████████████████| 68/68 [00:45<00:00,  1.48sarga/s]


✅ Completed Kanda 5 successfully!


📖 Starting Kanda 6 having 131 Sargas


Kanda 6: 100%|████████████████████████████████████████████████████████████████████| 131/131 [02:17<00:00,  1.05s/sarga]

✅ Completed Kanda 6 successfully!


🎉 Completed fetching Valmiki Ramayana!
Data saved to: valmiki_ramayana1.txt


In [ ]:
# Import required libraries
import re  # For using regular expressions to detect and extract structured patterns
import pandas as pd  # For handling tabular data and exporting it to CSV

# Initialize an empty list to store the final structured data
data = []

# Variables to hold the current Kanda and Sarga while parsing
current_kanda = None
current_sarga = None

# Open the previously scraped text file for reading
with open('valmiki_ramayana1.txt', 'r', encoding='utf-8') as file:
    prev_english = ""  # To temporarily hold English text when Sanskrit is missing

    # Read the file line by line
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespace

        # Detect the line indicating a new Kanda and Sarga using regex pattern
        kanda_sarga_match = re.match(r'###\s*Kanda\s*(\d+)\s*-\s*Sarga\s*(\d+)\s*###', line)
        if kanda_sarga_match:
            # If matched, update the current Kanda and Sarga context
            current_kanda = int(kanda_sarga_match.group(1))
            current_sarga = int(kanda_sarga_match.group(2))
            continue  # Skip processing this line further

        if not line:
            continue  # Skip empty lines

        # Use regex to split Sanskrit and English parts:
        # It groups Devanagari (Sanskrit) characters separately from English/alphanumeric characters
        parts = re.findall(r'([\u0900-\u097F:|।॥;()\'\-\s]+|[a-zA-Z0-9 .;\-:()\']+)', line)

        temp = []  # Temporary list to store pairs of [Sanskrit, English]

        # Iterate over each detected part
        for part in parts:
            part = part.strip()
            if not part:
                continue

            # If the part contains Devanagari characters, it's Sanskrit
            if re.search('[\u0900-\u097F]', part):
                part = part.lstrip(',')  # Clean any leading commas
                temp.append([part, ""])  # Add Sanskrit word with placeholder for English
            else:
                # If it's English and there's already a Sanskrit word waiting
                if temp:
                    if temp[-1][1] == "":
                        temp[-1][1] = part  # First English meaning for that Sanskrit word
                    else:
                        temp[-1][1] += " " + part  # Append additional meaning if it continues
                else:
                    # If no Sanskrit is present, store English in buffer for next valid pair
                    prev_english += " " + part

        # Add all collected [Sanskrit, English] word pairs to the main data list
        for sanskrit, english in temp:
            data.append([current_kanda, current_sarga, sanskrit, english.strip()])

        # If the line contained only English and not Sanskrit,
        # append this English text to the last entry's English column
        if not temp and prev_english:
            if data:
                data[-1][3] += " " + prev_english.strip()
            prev_english = ""

# Convert the list of data into a pandas DataFrame for structured tabular storage
df = pd.DataFrame(data, columns=["Kanda", "Sarga", "Sanskrit", "English"])

# Export the DataFrame to a CSV file with UTF-8 encoding
df.to_csv('clean_sanskrit_english.csv', index=False, encoding='utf-8-sig')

# Final confirmation print
print("CSV created successfully: clean_sanskrit_english.csv")


CSV created successfully: clean_sanskrit_english.csv


In [ ]:
# Import necessary libraries
import pandas as pd  # For working with tabular data
import re  # For regular expressions used in phonetic formatting
from indic_transliteration import sanscript  # For specifying script formats
from indic_transliteration.sanscript import transliterate  # For transliteration between scripts

# Load the cleaned Sanskrit-English word dataset
df = pd.read_csv('clean_sanskrit_english.csv')

# ----------------------------------------
# Function: transliterate_to_iast
# Purpose: Convert Devanagari script to IAST (International Alphabet of Sanskrit Transliteration)
# Input: Sanskrit string in Devanagari script
# Output: Corresponding IAST string
# ----------------------------------------
def transliterate_to_iast(text):
    return transliterate(text, sanscript.DEVANAGARI, sanscript.IAST)

# ----------------------------------------
# Function: generate_phonetics
# Purpose: Generate readable phonetic representation from IAST transliteration
# Input: IAST string (from previous function)
# Output: Pronunciation-friendly string with syllables hyphenated
# ----------------------------------------
def generate_phonetics(iast_text):
    phonetic = iast_text.lower()  # Convert to lowercase for uniformity

    # Replace IAST diacritics and complex sounds with English-friendly equivalents
    replacements = [
        ('kh', 'kh'), ('gh', 'gh'), ('ch', 'chh'), ('jh', 'jh'),
        ('ṭh', 'th'), ('ḍh', 'dh'), ('th', 'th'), ('dh', 'dh'),
        ('ph', 'ph'), ('bh', 'bh'),
        ('ā', 'aa'), ('ī', 'ee'), ('ū', 'oo'), ('ṛ', 'ri'), ('ṝ', 'ree'),
        ('ai', 'ai'), ('au', 'au'),
        ('ṅ', 'ng'), ('ñ', 'ny'), ('ṭ', 't'), ('ḍ', 'd'), ('ṇ', 'n'),
        ('ś', 'sh'), ('ṣ', 'sh'), ('ḥ', 'h'), ('ṃ', 'm')
    ]

    # Apply all replacements sequentially
    for src, tgt in replacements:
        phonetic = phonetic.replace(src, tgt)

    # Add hyphens after each Sanskrit consonant for syllable separation
    phonetic = re.sub(r'([kgcjṭḍtdnpbmyrlvsh])', r'\1-', phonetic)

    # Remove trailing hyphen if added at the end
    phonetic = phonetic.rstrip('-')

    return phonetic

# Initialize an empty list to store final output rows
final_output = []

# Loop over each row in the DataFrame
for _, row in df.iterrows():
    sanskrit = str(row['Sanskrit']).strip()  # Get Sanskrit word
    english = str(row['English']).strip()  # Get English meaning

    # Generate IAST transliteration from Sanskrit
    translit = transliterate_to_iast(sanskrit)

    # Generate phonetic breakdown from the IAST transliteration
    phonetic = generate_phonetics(translit)

    # Append the full row to the final output list
    final_output.append([sanskrit, english, translit, phonetic])

# Convert final list into a DataFrame
output_df = pd.DataFrame(final_output, columns=['Sanskrit', 'English', 'Transliteration', 'Phonetics'])

# Save the DataFrame to a new CSV file with UTF-8 encoding (for Devanagari support)
output_df.to_csv('final_sanskrit_output.csv', index=False, encoding='utf-8-sig')

# Confirmation message
print("File generated successfully: final_sanskrit_output.csv")


File generated successfully: final_sanskrit_output.csv


In [ ]:
import pandas as pd  # Used for working with CSV and tabular data
import stanza  # Library for NLP tasks, supports Sanskrit too

# Download the Sanskrit language model (only needs to be done once)
stanza.download('sa')

# Set up the NLP pipeline with tokenization and POS tagging
# We set tokenize_no_ssplit=True to keep each input as a single sentence
nlp = stanza.Pipeline(lang='sa', processors='tokenize,pos', tokenize_no_ssplit=True)

# Read the input CSV which contains Sanskrit words
df = pd.read_csv('final_sanskrit_phonetics.csv')

# Extract just the Sanskrit column as a list
sanskrit_words = df['Sanskrit'].tolist()

# Define how to interpret each POS tag syntactically
syntactic_map = {
    'NOUN': 'Subject/Object',
    'VERB': 'Action/State',
    'ADJ': 'Describes Noun',
    'ADV': 'Describes Verb/Adjective',
    'PRON': 'Pronoun Usage',
    'ADP': 'Preposition/Postposition',
    'DET': 'Determiner',
    'NUM': 'Numerical Info',
    'AUX': 'Auxiliary Verb',
    'PART': 'Particle',
    'SCONJ': 'Subordinating Conjunction',
    'CCONJ': 'Coordinating Conjunction',
    'INTJ': 'Interjection',
    'SYM': 'Symbol',
    'PUNCT': 'Punctuation'
}

# Define a dictionary for semantic meanings (manually curated)
semantic_map = {
    'रामः': 'Person - Lord Ram',
    'गच्छति': 'Verb - To Go',
    'फलम्': 'Object - Fruit',
    'गुरुः': 'Person - Teacher',
    'पुस्तकम्': 'Object - Book'
}

# Prepare a list to collect results
output = []

# Loop over each Sanskrit word
for word in sanskrit_words:
    doc = nlp(word)  # Run the NLP pipeline to get POS tagging
    for sent in doc.sentences:  # Iterate over sentence objects (usually one)
        for token in sent.words:  # Iterate over word tokens
            pos = token.upos  # Get the universal part-of-speech tag
             # Map to syntactic function
             # Get meaning if available

            # Store everything in a dictionary
            output.append({
                'Sanskrit Word': word,
                'POS Tag': pos,
                
            })

# Convert the output list into a DataFrame
output_df = pd.DataFrame(output)

# Save the final result to a CSV file
output_df.to_csv('sanskrit_pos_syntactic_semantic.csv', index=False)

# Let the user know the file has been created
print("CSV Generated Successfully!")
